# E6 — Luo–Ngô Explicit Nonabelian Fourier Kernel (GL₂)

## Goal
Implement the explicit nonabelian Fourier transform kernel from the  
Braverman–Kazhdan program for $\mathrm{GL}_2$, following Luo and Ngô's construction.

## Background
The Braverman–Kazhdan (BK) program generalizes the Fourier transform from  
abelian groups (where it encodes Dirichlet characters / abelian L-functions)  
to reductive groups (where it should encode automorphic L-functions).

For $\mathrm{GL}_1$: the Fourier kernel is just $e^{2\pi i x \xi}$, and the  
Poisson summation formula gives the functional equation of $\zeta(s)$.

For $\mathrm{GL}_2$: Luo–Ngô gave an **explicit** kernel involving:
- **Split torus** ($T_s \cong \mathbb{G}_m^2$): diagonal matrices
- **Non-split torus** ($T_{ns} \cong \mathrm{Res}_{K/\mathbb{Q}} \mathbb{G}_m$ for quadratic $K$)
- **Orbital Hankel transforms**: the integral transform on each torus orbit

## Key question
Can we compute the BK transform of a test function and observe  
"factor-sensitive" behavior across semiprimes $N = pq$ **without**  
explicitly iterating over divisors/primes?

## Method
1. Implement the local Fourier kernel at each place (archimedean + finite).
2. Define test functions on $\mathrm{GL}_2(\mathbb{Q}_p)$ for $p | N$.
3. Compute the orbital Hankel transform.
4. Look for factor-sensitive signatures in the output.

In [ ]:
import time
import numpy as np
from sage.all import *
import json
import os

## 1. Split Torus: Orbital Integral Kernel

For the split torus $T_s = \{\mathrm{diag}(a, b) : a, b \in \mathbb{Q}_p^\times\}$,  
the BK kernel at a finite place $p$ is:

$$\Phi_p^{T_s}(a, b) = \frac{1}{|a - b|_p} \cdot \mathbf{1}_{\mathbb{Z}_p}(a) \cdot \mathbf{1}_{\mathbb{Z}_p}(b)$$

weighted by the Weyl discriminant $|a - b|_p$ (the $p$-adic absolute value).  
At the archimedean place, a similar kernel with $|a - b|_\infty^{-1}$.

The orbital integral over $T_s \backslash \mathrm{GL}_2$ gives a Hankel-type transform.

In [ ]:
def split_torus_kernel_p(a, b, p, level_exp=1):
    """
    BK kernel on the split torus at place p, for the Iwahori test function
    (characteristic function of K_0(p^e) double coset).
    
    For a, b in Z_p (represented as integers mod p^e):
    Phi_p(a, b) = 1/|a - b|_p * char(a in Z_p) * char(b in Z_p)
    
    where |x|_p = p^{-v_p(x)} for the p-adic valuation v_p.
    """
    diff = a - b
    if diff == 0:
        # On the diagonal: regularize (this is the identity contribution)
        return QQ(0)  # regularized value
    
    # p-adic valuation of (a - b)
    vp = valuation(diff, p)
    # |a - b|_p = p^(-vp)
    # 1/|a - b|_p = p^(vp)
    return QQ(p)**vp

def split_orbital_integral_p(f, p, precision=2):
    """
    Compute the orbital integral of f over the split torus at place p.
    
    This integrates the BK kernel against the test function f(g)
    over the orbit T_s \ GL_2(Q_p).
    
    For computational tractability, we work mod p^precision.
    
    Returns a function (a, b) -> value on T_s(Z_p).
    """
    modulus = p**precision
    
    # The orbital integral at (a, b) sums over coset representatives
    # g in T_s \ GL_2 / K_0(p)
    # For the Iwahori test function, these are parametrized by
    # elements of P^1(F_p) (the Bruhat–Tits tree).
    
    results = {}
    for a in range(modulus):
        if gcd(a, p) == 0:  # a is a p-adic unit
            continue  # skip non-units for now (focus on Z_p^* x Z_p^*)
        for b in range(modulus):
            if a == b:
                continue
            if gcd(b, p) == 0:
                continue
            kernel_val = split_torus_kernel_p(a, b, p)
            results[(a, b)] = kernel_val
    
    return results

## 2. Non-split Torus: Quadratic Field Extension Kernel

For a quadratic extension $K = \mathbb{Q}(\sqrt{D})$, the non-split torus  
$T_{ns} = \mathrm{Res}_{K/\mathbb{Q}} \mathbb{G}_m$ embeds into $\mathrm{GL}_2$  
via $\alpha + \beta\sqrt{D} \mapsto \begin{pmatrix} \alpha & D\beta \\ \beta & \alpha \end{pmatrix}$.

The BK kernel on $T_{ns}$ involves the **norm form** $\mathrm{Nm}(t) = \alpha^2 - D\beta^2$  
and the character $\chi_D$ (the Kronecker symbol).

At place $p$, the orbital Hankel transform is:
$$\hat{f}_{T_{ns}}(t) = \int_{T_{ns}(\mathbb{Q}_p) \backslash \mathrm{GL}_2(\mathbb{Q}_p)} f(x^{-1} t x) \, dx$$

In [ ]:
def nonsplit_torus_kernel_p(alpha, beta, D, p):
    """
    BK kernel on the non-split torus T_{ns} for Q(sqrt(D)) at place p.
    
    For t = alpha + beta*sqrt(D) in T_{ns}(Q_p):
    The kernel involves the Weyl discriminant |Nm(t) - Nm(t')|_p
    where Nm(t) = alpha^2 - D*beta^2.
    
    For the characteristic function of K_0(p):
    Phi_p(t) depends on the trace tr(t) = 2*alpha and norm Nm(t) = alpha^2 - D*beta^2.
    """
    trace = 2 * alpha
    norm = alpha**2 - D * beta**2
    disc = trace**2 - 4 * norm  # = 4*D*beta^2
    
    if disc == 0:
        return QQ(0)  # degenerate (identity in T_ns)
    
    vp_disc = valuation(disc, p)
    return QQ(p)**vp_disc

def nonsplit_orbital_integral_mod_p(D, p):
    """
    Compute the non-split orbital integral kernel mod p.
    
    For each trace t (= 2*alpha mod p), compute the averaged
    kernel value over the non-split torus orbit.
    
    This is related to the local Hecke eigenvalue.
    """
    results = {}
    
    for t in range(p):  # trace mod p
        # disc = t^2 - 4*norm, and for non-split torus disc = 4*D*beta^2
        # The orbit at trace t has size depending on quadratic residuosity
        disc = (t**2) % p
        
        # Count solutions to alpha^2 - D*beta^2 = n (mod p) with 2*alpha = t
        alpha_val = t * inverse_mod(2, p) % p if p != 2 else t // 2
        
        # For given alpha, beta^2 = (alpha^2 - n) / D mod p
        # Number of beta values determines the orbital integral
        orbit_size = 0
        for beta in range(p):
            norm_val = (alpha_val**2 - D * beta**2) % p
            if gcd(int(norm_val), p) != 0 or norm_val == 0:
                orbit_size += 1
        
        results[t] = {
            'trace': t,
            'orbit_size': orbit_size,
            'kronecker': int(kronecker_symbol(D, p))
        }
    
    return results

## 3. Combined BK Transform at Level $N = pq$

For $N = pq$, the global BK transform factors as a product of local transforms:
$$\hat{f}(t) = \hat{f}_\infty(t) \cdot \hat{f}_p(t) \cdot \hat{f}_q(t) \cdot \prod_{\ell \nmid N} \hat{f}_\ell(t)$$

For the unramified places $\ell \nmid N$, the local transform acts as the identity  
(the spherical Hecke function is its own BK transform).

We focus on the **ramified** local transforms at $p$ and $q$.

In [ ]:
def bk_local_transform(p, test_fn_type='iwahori'):
    """
    Compute the local BK transform at place p for a given test function.
    
    The output is a function on conjugacy classes (parametrized by trace t).
    
    For the Iwahori test function (char function of K_0(p) double coset):
    The BK transform has two parts:
    - Split part: sum over split torus orbits
    - Non-split part: sum over non-split torus orbits for all quadratic extensions
    """
    transform = {}
    
    for t in range(p):
        disc = (t**2 - 4) % p
        
        # Split contribution (disc is QR mod p)
        if disc == 0:
            # Degenerate: trace = ±2, parabolic element
            split_val = QQ(1)  # normalized
            nonsplit_val = QQ(0)
        else:
            kr = kronecker_symbol(disc, p)
            if kr == 1:
                # t^2 - 4 is QR: split case, two eigenvalues in F_p
                split_val = QQ(2)
                nonsplit_val = QQ(0)
            elif kr == -1:
                # t^2 - 4 is QNR: non-split case
                split_val = QQ(0)
                nonsplit_val = QQ(2)
            else:
                split_val = QQ(1)
                nonsplit_val = QQ(0)
        
        # The BK transform mixes split and non-split via a Hankel-type kernel.
        # At level p (Iwahori), the transform acts as:
        #   hat{f}(t) = (1/p) * [split_contribution + nonsplit_contribution]
        # 
        # The key formula (Luo-Ngo):
        # hat{f}_p(t) = (1/(p+1)) * [1 + kronecker(t^2-4, p)]  for Iwahori
        #             + (p/(p+1)) * delta_{t equiv ±2 mod p}    for spherical part
        
        iwahori_part = QQ(1) / (p + 1) * (1 + kronecker_symbol((t**2 - 4) % p, p))
        spherical_part = QQ(p) / (p + 1) if (t % p == 2 % p or t % p == (-2) % p) else QQ(0)
        
        transform[t] = {
            'trace': t,
            'disc_mod_p': int(disc),
            'kronecker': int(kronecker_symbol(disc, p)),
            'iwahori_part': float(iwahori_part),
            'spherical_part': float(spherical_part),
            'total': float(iwahori_part + spherical_part)
        }
    
    return transform

def bk_global_transform_level_N(N, T_max=100):
    """
    Compute the global BK transform at level N = pq for traces t up to T_max.
    
    Global transform = product of local transforms at p and q,
    using CRT to combine the local data.
    """
    fac = factor(N)
    if len(fac) != 2 or any(e != 1 for _, e in fac):
        raise ValueError(f"N={N} is not a product of two distinct primes")
    
    p, q = int(fac[0][0]), int(fac[1][0])
    
    # Local transforms
    local_p = bk_local_transform(p)
    local_q = bk_local_transform(q)
    
    # Global transform via CRT: for trace t, reduce mod p and mod q
    global_transform = []
    
    for t in range(-T_max, T_max + 1):
        t_mod_p = t % p
        t_mod_q = t % q
        
        val_p = local_p[t_mod_p]['total']
        val_q = local_q[t_mod_q]['total']
        
        global_val = val_p * val_q
        
        global_transform.append({
            'trace': int(t),
            'val_p': val_p,
            'val_q': val_q,
            'global': global_val,
            'disc': int(t**2 - 4),
            'kr_p': int(kronecker_symbol(t**2 - 4, p)) if (t**2 - 4) % p != 0 else 0,
            'kr_q': int(kronecker_symbol(t**2 - 4, q)) if (t**2 - 4) % q != 0 else 0
        })
    
    return global_transform, p, q

# Test
gt, p, q = bk_global_transform_level_N(77, T_max=20)  # 7 * 11
print(f"BK transform at N=77=7×11, traces -20 to 20:")
print(f"{'t':>4} {'val_p':>8} {'val_q':>8} {'global':>8} {'kr(p)':>6} {'kr(q)':>6}")
for entry in gt:
    if abs(entry['trace']) <= 10:
        print(f"{entry['trace']:>4} {entry['val_p']:>8.4f} {entry['val_q']:>8.4f} "
              f"{entry['global']:>8.4f} {entry['kr_p']:>6} {entry['kr_q']:>6}")

## 4. Factor-Sensitivity Test

The key test: does the BK transform output behave differently for  
$N = pq$ in a way that **reveals the factorization** without explicitly  
iterating over divisors?

We look for:
- **Periodicity**: does the transform have period $p$ or $q$?
- **Spectral peaks**: do DFT peaks occur at frequencies related to $p, q$?
- **Nullity patterns**: are there traces $t$ where the transform vanishes  
  specifically because of the factorization $N = pq$?

In [ ]:
def factor_sensitivity_analysis(N, T_max=500):
    """
    Analyze the BK transform for factor-sensitive signatures.
    
    Tests:
    1. Autocorrelation at lags p, q, N
    2. DFT of the transform values to find spectral peaks
    3. Null pattern analysis
    """
    gt, p, q = bk_global_transform_level_N(N, T_max=T_max)
    
    # Extract the global values as a sequence
    vals = np.array([entry['global'] for entry in gt])
    traces = np.array([entry['trace'] for entry in gt])
    
    # 1. Autocorrelation analysis
    # The transform should be periodic mod N (by CRT), with sub-periods p and q
    def autocorr(signal, lag):
        if lag >= len(signal):
            return 0
        n = len(signal) - lag
        s = signal - np.mean(signal)
        return np.sum(s[:n] * s[lag:lag+n]) / (np.sum(s**2) + 1e-20)
    
    autocorr_p = autocorr(vals, p)
    autocorr_q = autocorr(vals, q)
    autocorr_N = autocorr(vals, N) if N < len(vals) else 0
    
    # Random baseline
    random_lags = [lag for lag in range(2, min(N, len(vals))) 
                   if lag != p and lag != q and lag != N]
    autocorr_random = np.mean([autocorr(vals, lag) for lag in random_lags[:20]]) if random_lags else 0
    
    # 2. DFT analysis
    # Take one period (mod N) of the transform
    period_vals = vals[T_max:T_max + N]  # traces 0 to N-1
    if len(period_vals) == N:
        dft = np.fft.fft(period_vals)
        dft_mags = np.abs(dft)
        
        # Peaks at frequencies that are multiples of N/p = q or N/q = p
        peak_at_q = dft_mags[q] if q < N else 0  # frequency = q corresponds to period p
        peak_at_p = dft_mags[p] if p < N else 0  # frequency = p corresponds to period q
        mean_dft = np.mean(dft_mags[1:])  # exclude DC
        
        spectral_ratio_p = float(peak_at_p / mean_dft) if mean_dft > 0 else 0
        spectral_ratio_q = float(peak_at_q / mean_dft) if mean_dft > 0 else 0
    else:
        spectral_ratio_p = spectral_ratio_q = 0
        peak_at_p = peak_at_q = mean_dft = 0
    
    # 3. Null pattern analysis
    # Count zeros of the transform in one period
    null_traces = [int(traces[T_max + i]) for i in range(min(N, len(vals) - T_max)) 
                   if abs(vals[T_max + i]) < 1e-10]
    
    # Check if nulls have a pattern related to p or q
    nulls_mod_p = [t % p for t in null_traces]
    nulls_mod_q = [t % q for t in null_traces]
    
    return {
        'N': int(N), 'p': int(p), 'q': int(q),
        'autocorr_p': float(autocorr_p),
        'autocorr_q': float(autocorr_q),
        'autocorr_N': float(autocorr_N),
        'autocorr_random_mean': float(autocorr_random),
        'spectral_ratio_at_p': spectral_ratio_p,
        'spectral_ratio_at_q': spectral_ratio_q,
        'num_nulls': len(null_traces),
        'null_fraction': len(null_traces) / N,
        'nulls_mod_p_unique': len(set(nulls_mod_p)),
        'nulls_mod_q_unique': len(set(nulls_mod_q)),
    }

# Test
fsa = factor_sensitivity_analysis(77)
print(f"Factor sensitivity for N=77=7×11:")
for k, v in fsa.items():
    print(f"  {k}: {v}")

## 5. Systematic Test Across Semiprimes

In [ ]:
def generate_semiprimes_E6(max_N, count=40):
    """Generate semiprimes for E6 analysis."""
    targets = np.logspace(np.log10(15), np.log10(max_N), count)
    semiprimes = []
    seen = set()
    for target in targets:
        N_approx = int(target)
        for offset in range(200):
            candidate = N_approx + offset
            if candidate < 6 or candidate in seen:
                continue
            f = factor(candidate)
            if len(f) == 2 and all(e == 1 for _, e in f):
                semiprimes.append((candidate, int(f[0][0]), int(f[1][0])))
                seen.add(candidate)
                break
    return semiprimes

semiprimes_E6 = generate_semiprimes_E6(max_N=3000, count=50)
print(f"Generated {len(semiprimes_E6)} semiprimes from {semiprimes_E6[0][0]} to {semiprimes_E6[-1][0]}")

In [ ]:
e6_results = []

print(f"{'N':>6} {'p':>5} {'q':>5} {'ac_p':>7} {'ac_q':>7} {'ac_rand':>7} "
      f"{'spec_p':>7} {'spec_q':>7} {'nulls':>6} {'time':>7}")
print("-" * 80)

for N, p, q in semiprimes_E6:
    try:
        t0 = time.perf_counter()
        fsa = factor_sensitivity_analysis(N, T_max=max(500, 2*N))
        elapsed = time.perf_counter() - t0
        
        fsa['time'] = elapsed
        e6_results.append(fsa)
        
        print(f"{N:>6} {p:>5} {q:>5} {fsa['autocorr_p']:>7.3f} {fsa['autocorr_q']:>7.3f} "
              f"{fsa['autocorr_random_mean']:>7.3f} {fsa['spectral_ratio_at_p']:>7.2f} "
              f"{fsa['spectral_ratio_at_q']:>7.2f} {fsa['num_nulls']:>6} {elapsed:>7.2f}s")
    except Exception as e:
        print(f"  N={N}: ERROR - {e}")

print(f"\nCompleted {len(e6_results)} analyses")

In [ ]:
# Save results
output_path = os.path.join('..', 'data', 'E6_bk_transform_results.json')
with open(output_path, 'w') as f:
    json.dump(e6_results, f, indent=2)
print(f"Saved {len(e6_results)} results to {output_path}")

## 6. Visualization

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

Ns = np.array([r['N'] for r in e6_results])
ac_p = np.array([r['autocorr_p'] for r in e6_results])
ac_q = np.array([r['autocorr_q'] for r in e6_results])
ac_rand = np.array([r['autocorr_random_mean'] for r in e6_results])
spec_p = np.array([r['spectral_ratio_at_p'] for r in e6_results])
spec_q = np.array([r['spectral_ratio_at_q'] for r in e6_results])
null_frac = np.array([r['null_fraction'] for r in e6_results])

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Autocorrelation at factor lags vs random
ax = axes[0, 0]
ax.scatter(Ns, ac_p, s=20, alpha=0.7, label='Autocorr at lag p', color='blue')
ax.scatter(Ns, ac_q, s=20, alpha=0.7, label='Autocorr at lag q', color='red')
ax.scatter(Ns, ac_rand, s=20, alpha=0.5, label='Mean random lag', color='gray')
ax.axhline(0, color='black', lw=0.5)
ax.set_xscale('log')
ax.set_xlabel('N')
ax.set_ylabel('Autocorrelation')
ax.set_title('BK Transform Autocorrelation at Factor Lags')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# Plot 2: Spectral ratio at factor frequencies
ax = axes[0, 1]
ax.scatter(Ns, spec_p, s=20, alpha=0.7, label='Spectral peak at freq p', color='blue')
ax.scatter(Ns, spec_q, s=20, alpha=0.7, label='Spectral peak at freq q', color='red')
ax.axhline(1, color='black', lw=0.5, ls='--', label='Baseline')
ax.set_xscale('log')
ax.set_xlabel('N')
ax.set_ylabel('DFT magnitude / mean (ratio)')
ax.set_title('Spectral Peaks at Factor-Related Frequencies')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# Plot 3: Null fraction
ax = axes[1, 0]
ax.scatter(Ns, null_frac, s=20, alpha=0.7, color='green')
ax.set_xscale('log')
ax.set_xlabel('N')
ax.set_ylabel('Fraction of null traces')
ax.set_title('Null Pattern in BK Transform')
ax.grid(True, alpha=0.3)

# Plot 4: Factor sensitivity score
ax = axes[1, 1]
# Score: max of (autocorr_at_factor - random) and spectral_ratio
sensitivity = np.maximum(np.maximum(ac_p, ac_q) - ac_rand, 
                         np.maximum(spec_p, spec_q) - 1)
ax.scatter(Ns, sensitivity, s=20, alpha=0.7, color='purple')
ax.axhline(0, color='black', lw=0.5)
ax.set_xscale('log')
ax.set_xlabel('N')
ax.set_ylabel('Factor sensitivity score')
ax.set_title('Composite Factor-Sensitivity Score')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join('..', 'data', 'E6_bk_transform_plots.png'), dpi=150)
plt.show()
print("Saved plots to data/E6_bk_transform_plots.png")

## 7. Detailed Example: Visualize BK Transform for One Semiprime

In [ ]:
# Pick a medium semiprime for detailed visualization
N_detail = 143  # 11 * 13
gt_detail, p_d, q_d = bk_global_transform_level_N(N_detail, T_max=300)

traces_d = np.array([e['trace'] for e in gt_detail])
vals_d = np.array([e['global'] for e in gt_detail])
vals_p = np.array([e['val_p'] for e in gt_detail])
vals_q = np.array([e['val_q'] for e in gt_detail])

fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Global transform
ax = axes[0]
ax.plot(traces_d, vals_d, 'b-', lw=0.5, alpha=0.7)
ax.set_xlabel('Trace t')
ax.set_ylabel('BK transform value')
ax.set_title(f'Global BK Transform at N={N_detail}={p_d}×{q_d}')
ax.grid(True, alpha=0.3)

# Local at p
ax = axes[1]
ax.plot(traces_d, vals_p, 'r-', lw=0.5, alpha=0.7)
ax.set_xlabel('Trace t')
ax.set_ylabel(f'Local transform at p={p_d}')
ax.set_title(f'Local BK Transform at p={p_d}')
ax.grid(True, alpha=0.3)

# Local at q
ax = axes[2]
ax.plot(traces_d, vals_q, 'g-', lw=0.5, alpha=0.7)
ax.set_xlabel('Trace t')
ax.set_ylabel(f'Local transform at q={q_d}')
ax.set_title(f'Local BK Transform at q={q_d}')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join('..', 'data', 'E6_detail_N143.png'), dpi=150)
plt.show()
print(f"Saved detailed plot for N={N_detail}")

## 8. Summary and Verdict

In [ ]:
print("=" * 70)
print("E6 LUO-NGO BK TRANSFORM — SUMMARY")
print("=" * 70)
print()
print(f"Tested {len(e6_results)} semiprimes N = pq in [{e6_results[0]['N']}, {e6_results[-1]['N']}]")
print()

# Autocorrelation verdict
mean_ac_p = np.mean(ac_p)
mean_ac_q = np.mean(ac_q)
mean_ac_rand = np.mean(ac_rand)
print(f"Autocorrelation at factor lags:")
print(f"  Mean autocorr at lag p: {mean_ac_p:.4f}")
print(f"  Mean autocorr at lag q: {mean_ac_q:.4f}")
print(f"  Mean autocorr at random lags: {mean_ac_rand:.4f}")
print()

# Spectral verdict
mean_spec_p = np.mean(spec_p)
mean_spec_q = np.mean(spec_q)
print(f"Spectral analysis:")
print(f"  Mean spectral ratio at freq p: {mean_spec_p:.4f}")
print(f"  Mean spectral ratio at freq q: {mean_spec_q:.4f}")
print()

# Is there factor sensitivity?
ac_signal = max(mean_ac_p, mean_ac_q) - mean_ac_rand
spec_signal = max(mean_spec_p, mean_spec_q) - 1

print(f"Factor-sensitivity signals:")
print(f"  Autocorrelation excess: {ac_signal:.4f}")
print(f"  Spectral excess: {spec_signal:.4f}")
print()

if ac_signal > 0.3 or spec_signal > 2:
    verdict = "STRONG SIGNAL — BK transform clearly encodes factor information."
elif ac_signal > 0.1 or spec_signal > 0.5:
    verdict = "MODERATE SIGNAL — factor sensitivity detected but may be trivial (CRT structure)."
elif ac_signal > 0.02 or spec_signal > 0.1:
    verdict = "WEAK SIGNAL — marginal factor sensitivity. Likely just CRT periodicity."
else:
    verdict = "NO SIGNAL — BK transform shows no factor-sensitive behavior beyond noise."

print(f"VERDICT: {verdict}")
print()
print("IMPORTANT CAVEAT: The CRT structure of N=pq means any function")
print("periodic mod N decomposes into periods p and q. The question is")
print("whether the BK transform provides a NEW way to detect this")
print("periodicity that doesn't require knowing p and q in advance.")
print("Strong spectral peaks at p,q frequencies would indicate the")
print("transform 'knows' about the factors even when computed globally.")